# MAG Sample: Get PaperAuthorAffiliation data for an Affiliation

## Prerequisites

Complete these tasks before you begin this tutorial:

- Setting up provisioning of Microsoft Academic Graph to an Azure blob storage account. See [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning).
- Setting up Azure Databricks service. See [Set up Azure Databricks](https://docs.microsoft.com/academic-services/graph/get-started-setup-databricks).
- Install python library `plotly`, and `pycountry` on the cluster you want to run this tutorial.

## Gather the information

Before you begin, you should have these items of information:

- The name of your Azure Storage (AS) account containing MAG dataset from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The access key of your Azure Storage (AS) account from [Get Microsoft Academic Graph on Azure storage](https://docs.microsoft.com/academic-services/graph/get-started-setup-provisioning#note-azure-storage-account-name-and-primary-key).
- The name of the container in your Azure Storage (AS) account containing MAG dataset.

## Import notebooks

- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) samples/pyspark/MagClass.py under your working folder.
- [Import](https://docs.databricks.com/user-guide/notebooks/notebook-manage.html#import-a-notebook) this notebook under the same folder.

### Initialize storage account and container details

  | Variable  | Value | Description  |
  | --------- | --------- | --------- |
  | AzureStorageAccount | Replace **`<AzureStorageAccount>`** | This is the Azure Storage account containing MAG dataset. |
  | AzureStorageAccessKey | Replace **`<AzureStorageAccessKey>`** | This is the Access Key of the Azure Storage account. |
  | MagContainer | Replace **`<MagContainer>`** | This is the container name in Azure Storage account containing MAG dataset, usually in the form of mag-yyyy-mm-dd. |
  | OutputContainer | Replace **`<OutputContainer>`** | This is the container name in Azure Storage account where the output goes to, this container needs to be created before running this script. |

In [0]:
AzureStorageAccount = '<AzureStorageAccount>'
AzureStorageAccessKey = '<AzureStorageAccessKey>'
MagContainer = '<MagContainer>'
OutputContainer = '<OutputContainer>'

### Define MicrosoftAcademicGraph class

Run the MagClass notebook to define MicrosoftAcademicGraph class.

In [0]:
%run "./MagClass"

### Create a MicrosoftAcademicGraph instance to access MAG dataset
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer.

In [0]:
mag = MicrosoftAcademicGraph(account=AzureStorageAccount, key=AzureStorageAccessKey, container=MagContainer)

### Create a AzureStorageUtil to access other Azure Storage files
Use account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer.

In [0]:
asu = AzureStorageUtil(account=AzureStorageAccount, key=AzureStorageAccessKey, container=OutputContainer)

### Load Affiliation data for the input organization from previous output

In [0]:
affiliations = asu.load('Affiliation.tsv')

### Load PaperAuthorAffiliations

In [0]:
paperAuthorAffiliations = mag.getDataframe('PaperAuthorAffiliations')

# Peek result
paperAuthorAffiliations.show(10)

+-------+----------+-------------+--------------------+--------------------+--------------------+
PaperId| AuthorId|AffiliationId|AuthorSequenceNumber| OriginalAuthor| OriginalAffiliation|
+-------+----------+-------------+--------------------+--------------------+--------------------+
 9|2632942543| null| 1|Victoriano Perruc...| null|
 15| 199142497| null| 1| Robert Münscher|Heidelberg, Deuts...|
 15| 680395887| null| 2| Julia Hormuth|Reutlingen, Deuts...|
 23|1243978490| 79576946| 1| Eric T Stoopler|Department of Ora...|
 23|2582258949| null| 3| Arthur S Kuperstein| null|
 23|2582736345| null| 2| Ying Wai Sia| null|
 58|2641488431| null| 1| I. Uray| null|
 58|2693978806| null| 2| P. Kovacs| null|
 79|2662843304| null| 1| K. Suresh| null|
 79|2683440697| null| 2| T. S. J. Norman| null|
+-------+----------+-------------+--------------------+--------------------+--------------------+
only showing top 10 rows

### Filter PaperAuthorAffiliations by AffiliationId

In [0]:
orgPaperAuthorAffiliation = paperAuthorAffiliations \
    .join(affiliations, paperAuthorAffiliations.AffiliationId == affiliations.AffiliationId, 'inner') \
    .select(paperAuthorAffiliations.PaperId, paperAuthorAffiliations.AuthorId, \
            affiliations.AffiliationId, paperAuthorAffiliations.AuthorSequenceNumber)

# Peek result
orgPaperAuthorAffiliation.show(10)

# Count number of rows in result
print('Number of rows in PaperAuthorAffiliation: {}'.format(orgPaperAuthorAffiliation.count()))

+-------+----------+-------------+--------------------+
PaperId| AuthorId|AffiliationId|AuthorSequenceNumber|
+-------+----------+-------------+--------------------+
 15883| 135218249| 1290206253| 1|
 15883|2120803867| 1290206253| 2|
 15883|2128381254| 1290206253| 3|
 15883|2150137935| 1290206253| 4|
 73910|2279432523| 1290206253| 1|
 124417|2191432412| 1290206253| 1|
 649244|2152243552| 1290206253| 1|
 649502|2777639049| 1290206253| 1|
 786208|1957630529| 1290206253| 1|
 786208|2205570262| 1290206253| 2|
+-------+----------+-------------+--------------------+
only showing top 10 rows

Number of rows in PaperAuthorAffiliation: 221410

### Save PaperAuthorAffiliationRelationship.tsv

In [0]:
asu.save(orgPaperAuthorAffiliation, 'PaperAuthorAffiliationRelationship.tsv')